#  SNNbuilder MCP Demo: AI-Augmented Computational Neuroscience

This notebook demonstrates how to use SNNbuilder nodes through MCP (Model Context Protocol) servers with LLM agents for AI-augmented computational neuroscience.


## Step 1: Install Dependencies

First, let's install all the required packages:

In [1]:
# Install required packages
#!pip install -r requirements.txt

# Optional: Install NEST Simulator for full functionality
# Uncomment the line below if you want full NEST integration
# !pip install nest-simulator

## Step 2: Configure API Key

Set up your OpenAI API key for LLM interactions:

In [2]:
import json
import os

# Check if key.json exists
if not os.path.exists('key.json'):
    # Create key.json from template
    with open('key_template.json', 'r') as f:
        config = json.load(f)
    
    # Prompt user for API key
    #api_key = input("Please enter your OpenAI API key: ")
    #config['OPENAI_API_KEY'] = 
    
    # Save configuration
    with open('key.json', 'w') as f:
        json.dump(config, f, indent=2)
    
    print(" API key configured successfully!")
else:
    print(" key.json already exists")

# Verify configuration
with open('key.json', 'r') as f:
    config = json.load(f)

if config['OPENAI_API_KEY'] == 'your-openai-api-key-here':
    print("  Please update your API key in key.json")
else:
    print(f" Using model: {config['LLM_MODEL']}")

 key.json already exists
 Using model: gpt-4o-mini


## Step 3: Setup MCP Client

Now let's set up the MCP client to communicate with our SNNbuilder server:

In [3]:
import os, json, asyncio, sys
from typing import Dict, Any, List
from openai import OpenAI

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

def json_read(filename: str):
    if os.path.isfile(filename):
        with open(filename, "r", encoding="utf-8") as f:
            data = json.load(f)
        return data, True
    return {}, False

async def tail_file(path: str):
    """Monitor server log file for debugging."""
    try:
        with open(path, "r", encoding="utf-8") as f:
            f.seek(0, 2)  # jump to end
            while True:
                line = f.readline()
                if not line:
                    await asyncio.sleep(0.2)
                    continue
                print("[SNNBUILDER-SERVER]", line.rstrip())
    except FileNotFoundError:
        # file may not exist yet on first run
        for _ in range(25):
            await asyncio.sleep(0.2)
            try:
                with open(path, "r", encoding="utf-8") as f:
                    break
            except FileNotFoundError:
                pass
        # try again recursively if it appeared
        return await tail_file(path)

async def build_system_prompt(session) -> str:
    """Build system prompt with available tools."""
    tools = (await session.list_tools()).tools
    lines = [
        "You are an AI assistant specialized in computational neuroscience using SNNbuilder.",
        "You can help users create, configure, and execute neural models with biological parameters.",
        "Use tools when appropriate to help users with neuron modeling tasks.",
        "Always provide clear explanations of neuroscience concepts and parameter meanings.",
        "",
        "Available SNNbuilder tools:"
    ]
    for t in tools:
        desc = (t.description or "").strip()
        # grab the first paragraph only for brevity
        first_para = desc.split("\n\n", 1)[0]
        lines.append(f"- {t.name}: {first_para}")
    lines.append("")
    lines.append("Use tool parameters exactly as defined in the provided tool schemas.")
    return "\n".join(lines)

# Load API configuration
api_data, ok = json_read("key.json")
if not ok:
    raise FileNotFoundError("key.json not found. Please run the previous cell first.")

PROVIDER_BASE = api_data.get("PROVIDER_BASE", "").strip()
API_KEY       = api_data.get("OPENAI_API_KEY", "").strip()
MODEL         = api_data.get("LLM_MODEL", "gpt-4o-mini").strip()
SERVER_PATH   = api_data.get("MCP_SERVER", "snnbuilder_server.py").strip()

def make_client() -> OpenAI:
    if PROVIDER_BASE:
        return OpenAI(base_url=PROVIDER_BASE, api_key=API_KEY)
    return OpenAI(api_key=API_KEY)

def schema_from_mcp_tool(t) -> Dict[str, Any]:
    """Convert an MCP ToolDescriptor to OpenAI tool schema."""
    params = getattr(t, "inputSchema", None) or getattr(t, "input_schema", None)
    if not params:
        params = {"type": "object", "properties": {}, "additionalProperties": False}
    return {
        "type": "function",
        "function": {
            "name": t.name,
            "description": getattr(t, "description", "") or "",
            "parameters": params,
        },
    }

def extract_text_content(mcp_result) -> str:
    """Extract text content from MCP result."""
    parts = []
    for c in getattr(mcp_result, "content", []) or []:
        text = getattr(c, "text", None)
        if text is not None:
            parts.append(text)
    return "\n".join(parts) if parts else ""

async def build_openai_tools(session: ClientSession) -> List[Dict[str, Any]]:
    """Build OpenAI tools from MCP session."""
    tools_resp = await session.list_tools()
    return [schema_from_mcp_tool(t) for t in tools_resp.tools]

print(" MCP client setup complete!")

 MCP client setup complete!


## Step 4: Create the Main Chat Function

This function handles the communication between the LLM and the SNNbuilder MCP server:

In [6]:
async def run_snnbuilder_chat(user_msg: str, show_details: bool = True) -> str:
    """Run a chat session with SNNbuilder MCP server."""
    client = make_client()

    # Spawn the SNNbuilder MCP server over stdio and open a session
    params = StdioServerParameters(command=sys.executable, args=[SERVER_PATH])

    # start tail task before connecting (for debugging)
    log_task = asyncio.create_task(tail_file("snnbuilder_server.log")) if show_details else None
    
    async with stdio_client(params) as (read_stream, write_stream):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()

            openai_tools = await build_openai_tools(session)

            if show_details:
                print(" Available SNNbuilder tools:")
                for tool in openai_tools:
                    print(f"   • {tool['function']['name']}")
                print()

            # Build system prompt
            system_prompt = await build_system_prompt(session)
            
            if show_details:
                print(" System prompt configured")
                print("\n" + "="*60 + "\n")

            messages: List[Dict[str, Any]] = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_msg},
            ]

            for iteration in range(5):
                if show_details:
                    print(f" LLM Iteration {iteration + 1}:")
                
                chat = client.chat.completions.create(
                    model=MODEL,
                    messages=messages,
                    tools=openai_tools,
                    tool_choice="auto",
                )
                choice = chat.choices[0]
                finish = choice.finish_reason

                if finish == "tool_calls" and choice.message.tool_calls:
                    if show_details:
                        print(f"    Calling {len(choice.message.tool_calls)} tool(s):")
                    
                    tool_messages = []
                    for call in choice.message.tool_calls:
                        name = call.function.name
                        args = json.loads(call.function.arguments or "{}")
                        if show_details:
                            print(f"      • {name}({', '.join(f'{k}={v}' for k, v in list(args.items())[:3])}{'...' if len(args) > 3 else ''})")
                        
                        result = await session.call_tool(name, args)
                        tool_result = extract_text_content(result)
                        
                        tool_messages.append({
                            "role": "tool",
                            "tool_call_id": call.id,
                            "name": name,
                            "content": tool_result,
                        })
                    
                    messages = messages + [choice.message] + tool_messages
                    continue

                if show_details:
                    print("\n Final LLM Response:")
                    print("="*60)
                return choice.message.content or ""

    # stop the tailer
    if log_task:
        log_task.cancel()
        try:
            await log_task
        except asyncio.CancelledError:
            pass
        
    return ""

# Notebook-friendly entry point
def run_snnbuilder_demo(prompt: str, show_details: bool = True):
    """Run SNNbuilder demo with LLM interaction."""
    # Jupyter often has an existing event loop. Use nest_asyncio to allow nested runs.
    import nest_asyncio
    nest_asyncio.apply()
    try:
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(run_snnbuilder_chat(prompt, show_details))
    except RuntimeError:
        # Fallback: create a new loop
        return asyncio.run(run_snnbuilder_chat(prompt, show_details))

print("Chat function ready!")
print("\n You can now use: run_snnbuilder_demo('your question here')")

Chat function ready!

 You can now use: run_snnbuilder_demo('your question here')


#  Interactive Demos

Now let's explore AI-augmented computational neuroscience! Each demo showcases different aspects of the SNNbuilder MCP integration.

## Demo 1: Learning About Neuron Creation

In [7]:
# Ask the LLM to explain neuron creation
response = run_snnbuilder_demo(
    "How do I create a neuron? What are the parameters and their meanings? "
    "Please explain it in a way that's educational for someone learning computational neuroscience."
)
print(response)

 Available SNNbuilder tools:
   • explain_neuron_creation
   • create_single_neuron
   • execute_single_neuron
   • generate_neuron_code
   • list_created_neurons

 System prompt configured


 LLM Iteration 1:
    Calling 1 tool(s):
      • explain_neuron_creation()
[SNNBUILDER-SERVER] 2025-09-30 06:04:30,063 [DEBUG] SNNbuilder SingleNeuron MCP server starting...
 LLM Iteration 2:

 Final LLM Response:
[SNNBUILDER-SERVER] 2025-09-30 06:04:32,919 [DEBUG] explain_neuron_creation() called
Creating a neuron model in computational neuroscience involves defining various biological and computational parameters that describe how the neuron behaves and interacts with its environment. The SNNbuilder allows you to create neuron models effectively by using a structured approach. Here's a comprehensive educational overview of how to create a neuron and the meanings of its parameters:

### Overview of Neuron Creation

1. **Create Node Instance**: You start by creating a single neuron node with a uni

## Demo 2: Creating a Pyramidal Neuron

In [8]:
# Ask the LLM to create a pyramidal neuron
response = run_snnbuilder_demo(
    "Can you create a Layer 2/3 pyramidal neuron for me with realistic biological parameters? "
    "Please explain why you chose those specific parameter values."
)
print(response)

 Available SNNbuilder tools:
   • explain_neuron_creation
   • create_single_neuron
   • execute_single_neuron
   • generate_neuron_code
   • list_created_neurons

 System prompt configured


 LLM Iteration 1:
    Calling 1 tool(s):
      • create_single_neuron(neuron_id=pyr_l23, name=Layer 2/3 Pyramidal Neuron, acronym=PyL23...)
[SNNBUILDER-SERVER] 2025-09-30 06:08:10,520 [DEBUG] SNNbuilder SingleNeuron MCP server starting...
 LLM Iteration 2:

 Final LLM Response:
[SNNBUILDER-SERVER] 2025-09-30 06:08:14,419 [DEBUG] create_single_neuron() called with neuron_id=pyr_l23
I've successfully created a Layer 2/3 pyramidal neuron with the following parameters:

### Configured Parameters:
- **Neuron ID:** pyr_l23
- **Name:** Layer 2/3 Pyramidal Neuron
- **Acronym:** PyL23
- **Cell Class:** Excitatory
- **Dendrite Extent:** 300.0 μm
- **Firing Rate (Resting):** Minimum 2.0 Hz, Maximum 8.0 Hz
- **NEST Model:** iaf_cond_exp
- **Threshold Potential (V_th):** -50.0 mV
- **Membrane Capacitance (C_m)

## Demo 3: Executing the Neuron

In [9]:
# Ask the LLM what to do next and execute the neuron
response = run_snnbuilder_demo(
    "What should I do next with the pyramidal neuron I created? "
    "Please execute it and show me what happens. Explain each step."
)
print(response)

 Available SNNbuilder tools:
   • explain_neuron_creation
   • create_single_neuron
   • execute_single_neuron
   • generate_neuron_code
   • list_created_neurons

 System prompt configured


 LLM Iteration 1:
    Calling 1 tool(s):
      • execute_single_neuron(neuron_id=pyr_l23)
[SNNBUILDER-SERVER] 2025-09-30 06:09:56,382 [DEBUG] SNNbuilder SingleNeuron MCP server starting...
 LLM Iteration 2:
    Calling 1 tool(s):
      • list_created_neurons()
[SNNBUILDER-SERVER] 2025-09-30 06:09:57,611 [DEBUG] execute_single_neuron() called with neuron_id=pyr_l23
 LLM Iteration 3:

 Final LLM Response:
[SNNBUILDER-SERVER] 2025-09-30 06:09:58,634 [DEBUG] list_created_neurons() called
It seems that there are no neurons created in the current session, which is why I couldn't execute the pyramidal neuron with the ID "pyr_l23." 

### Next Steps:

1. **Create the Neuron**:
   Since there are no neurons in the session yet, you'll need to create one. You can specify parameters such as the neuron's name, 

## Demo 4: Generating Python Code

In [10]:
# Ask the LLM to generate standalone Python code
response = run_snnbuilder_demo(
    "Can you generate standalone Python code for the pyramidal neuron that I can run independently? "
    "Please explain what the code does and how to use it."
)
print(response)

 Available SNNbuilder tools:
   • explain_neuron_creation
   • create_single_neuron
   • execute_single_neuron
   • generate_neuron_code
   • list_created_neurons

 System prompt configured


 LLM Iteration 1:
    Calling 1 tool(s):
      • generate_neuron_code(neuron_id=pyr_l23, code_format=python)
[SNNBUILDER-SERVER] 2025-09-30 06:10:59,811 [DEBUG] SNNbuilder SingleNeuron MCP server starting...
 LLM Iteration 2:
    Calling 1 tool(s):
      • list_created_neurons()
[SNNBUILDER-SERVER] 2025-09-30 06:11:01,712 [DEBUG] generate_neuron_code() called with neuron_id=pyr_l23
 LLM Iteration 3:

 Final LLM Response:
[SNNBUILDER-SERVER] 2025-09-30 06:11:03,143 [DEBUG] list_created_neurons() called
It appears that no neurons have been created in the current session yet. Before generating standalone Python code for the pyramidal neuron (or any neuron), we need to create it first.

### Creating a Pyramidal Neuron

To proceed, I can help you create a pyramidal neuron with specific parameters. Here

## Demo 5: Creating an Inhibitory Neuron

In [11]:
# Ask the LLM to create an inhibitory neuron
response = run_snnbuilder_demo(
    "Now create a fast-spiking inhibitory interneuron with appropriate parameters. "
    "Make it different from the pyramidal neuron with faster dynamics. "
    "Explain the biological differences between excitatory and inhibitory neurons."
)
print(response)

 Available SNNbuilder tools:
   • explain_neuron_creation
   • create_single_neuron
   • execute_single_neuron
   • generate_neuron_code
   • list_created_neurons

 System prompt configured


 LLM Iteration 1:
    Calling 1 tool(s):
      • create_single_neuron(neuron_id=fast_spiking_int, name=Fast-Spiking Inhibitory Interneuron, acronym=FSI...)
[SNNBUILDER-SERVER] 2025-09-30 06:11:49,629 [DEBUG] SNNbuilder SingleNeuron MCP server starting...
 LLM Iteration 2:

 Final LLM Response:
[SNNBUILDER-SERVER] 2025-09-30 06:12:04,179 [DEBUG] create_single_neuron() called with neuron_id=fast_spiking_int
The fast-spiking inhibitory interneuron has been successfully created with the specified parameters. Here’s a summary of the properties of the neuron:

### Fast-Spiking Inhibitory Interneuron Properties
- **Name**: Fast-Spiking Inhibitory Interneuron
- **Acronym**: FSI
- **Cell Class**: Inhibitory
- **Dendrite Extent**: 200.0 μm
- **Resting Firing Rate**: 20.0 - 100.0 Hz
- **NEST Model**: iaf_psc

## Demo 6: Managing Multiple Neurons

In [12]:
# Ask the LLM to show all created neurons
response = run_snnbuilder_demo(
    "Show me all the neurons I've created so far and their status. "
    "Execute any that haven't been executed yet and compare their properties."
)
print(response)

 Available SNNbuilder tools:
   • explain_neuron_creation
   • create_single_neuron
   • execute_single_neuron
   • generate_neuron_code
   • list_created_neurons

 System prompt configured


 LLM Iteration 1:
    Calling 1 tool(s):
      • list_created_neurons()
[SNNBUILDER-SERVER] 2025-09-30 06:13:01,376 [DEBUG] SNNbuilder SingleNeuron MCP server starting...
 LLM Iteration 2:

 Final LLM Response:
[SNNBUILDER-SERVER] 2025-09-30 06:13:03,567 [DEBUG] list_created_neurons() called
It appears that no neurons have been created yet in your session. If you'd like to proceed, I can help you create your first neuron. Would you like me to explain how to create a neuron, or do you have specific parameters in mind for the neuron you'd like to create?
[SNNBUILDER-SERVER] 2025-09-30 06:13:23,991 [DEBUG] SNNbuilder SingleNeuron MCP server starting...
[SNNBUILDER-SERVER] 2025-09-30 06:13:26,912 [DEBUG] create_single_neuron() called with neuron_id=custom_exc_neuron
[SNNBUILDER-SERVER] 2025-09-30 06:1

## Demo 7: Custom Neuron with Specific Parameters

In [13]:
# Ask the LLM to create a neuron with specific custom parameters
response = run_snnbuilder_demo(
    "Create a custom neuron with these specific parameters: "
    "threshold at -45 mV, membrane capacitance of 150 pF, "
    "time constant of 12 ms, and dendritic extent of 250 micrometers. "
    "Make it an excitatory neuron, execute it immediately, and generate the code."
)
print(response)

 Available SNNbuilder tools:
   • explain_neuron_creation
   • create_single_neuron
   • execute_single_neuron
   • generate_neuron_code
   • list_created_neurons

 System prompt configured


 LLM Iteration 1:
    Calling 1 tool(s):
      • create_single_neuron(neuron_id=custom_exc_neuron, name=Custom Excitatory Neuron, dendrite_extent=250...)
[SNNBUILDER-SERVER] 2025-09-30 06:13:23,991 [DEBUG] SNNbuilder SingleNeuron MCP server starting...
 LLM Iteration 2:
    Calling 2 tool(s):
      • execute_single_neuron(neuron_id=custom_exc_neuron)
[SNNBUILDER-SERVER] 2025-09-30 06:13:26,912 [DEBUG] create_single_neuron() called with neuron_id=custom_exc_neuron
      • generate_neuron_code(neuron_id=custom_exc_neuron, code_format=python)
 LLM Iteration 3:

 Final LLM Response:
[SNNBUILDER-SERVER] 2025-09-30 06:13:29,879 [DEBUG] execute_single_neuron() called with neuron_id=custom_exc_neuron
[SNNBUILDER-SERVER] 2025-09-30 06:13:29,889 [DEBUG] generate_neuron_code() called with neuron_id=custom_ex

## Demo 8: Educational Comparison

In [14]:
# Ask the LLM to compare different neuron types
response = run_snnbuilder_demo(
    "Can you explain the differences between the pyramidal neuron and inhibitory neuron I created? "
    "What are the key parameter differences and why are they important for their biological functions? "
    "How do these differences affect their role in neural circuits?"
)
print(response)

 Available SNNbuilder tools:
   • explain_neuron_creation
   • create_single_neuron
   • execute_single_neuron
   • generate_neuron_code
   • list_created_neurons

 System prompt configured


 LLM Iteration 1:

 Final LLM Response:
[SNNBUILDER-SERVER] 2025-09-30 06:14:23,227 [DEBUG] SNNbuilder SingleNeuron MCP server starting...
Pyramidal neurons and inhibitory neurons (often GABAergic interneurons) perform distinct roles in neural circuits, characterized by key differences in their structure and function. Let's look at the primary parameter differences and their biological significance:

### Key Parameter Differences

1. **Cell Class**
   - **Pyramidal Neuron**: Typically classified as "excitatory," these neurons release excitatory neurotransmitters like glutamate.
   - **Inhibitory Neuron**: Classified as "inhibitory," they primarily release GABA (gamma-aminobutyric acid), which hyperpolarizes target neurons.

2. **Firing Rate Resting**
   - **Pyramidal Neuron**: Usually has a higher

## Demo 9: Best Practices and Troubleshooting

In [15]:
# Ask the LLM for help with common issues
response = run_snnbuilder_demo(
    "What are some common mistakes when creating neurons? "
    "Can you give me best practices for choosing neuron parameters? "
    "What should I consider when modeling different types of neurons?"
)
print(response)

 Available SNNbuilder tools:
   • explain_neuron_creation
   • create_single_neuron
   • execute_single_neuron
   • generate_neuron_code
   • list_created_neurons

 System prompt configured


 LLM Iteration 1:

 Final LLM Response:
[SNNBUILDER-SERVER] 2025-09-30 06:24:14,075 [DEBUG] SNNbuilder SingleNeuron MCP server starting...
When creating neuron models, especially in computational neuroscience, there are several common mistakes and best practices to consider. Here are some insights:

### Common Mistakes When Creating Neurons

1. **Ignoring Biophysical Realism**: It's easy to choose parameters without considering their biological basis. Ensure the parameters you select reflect the properties of real neurons.

2. **Parameter Conflicts**: Some parameters are interdependent. For instance, selecting an unrealistic firing threshold (v_th) may not be compatible with your chosen membrane potential (v_reset).

3. **Overlooking Default Values**: Many tools provide default parameters. While t

## Demo 10: Complete Workflow Example

In [16]:
# Ask the LLM to demonstrate a complete workflow
response = run_snnbuilder_demo(
    "Show me a complete workflow: create a cortical neuron, execute it, "
    "generate the Python code, and explain what each step does. "
    "Make it educational for someone learning computational neuroscience. "
    "Include tips for extending this to build neural networks."
)
print(response)

 Available SNNbuilder tools:
   • explain_neuron_creation
   • create_single_neuron
   • execute_single_neuron
   • generate_neuron_code
   • list_created_neurons

 System prompt configured


 LLM Iteration 1:

 Final LLM Response:
[SNNBUILDER-SERVER] 2025-09-30 06:25:45,614 [DEBUG] SNNbuilder SingleNeuron MCP server starting...
Certainly! Let's go through a complete workflow to model a cortical neuron step-by-step using SNNbuilder. Each step will be explained in detail to help you understand the underlying concepts in computational neuroscience.

### Step 1: Create a Cortical Neuron

First, we need to create a cortical neuron. In this case, let's create a Layer 2/3 pyramidal neuron, which is typical in the cortex and exhibits both excitatory and inhibitory characteristics.

#### Code to Create the Neuron

```python
create_single_neuron(
    neuron_id="pyr_l23",
    name="Layer 2/3 Pyramidal Neuron",
    acronym="PyL23",
    cell_class="excitatory",
    dendrite_extent=300.0,
    firin

# Interactive Mode

Now you can ask your own questions! Use the cell below to interact directly with the SNNbuilder MCP system:

In [17]:
# Ask your own questions here!
your_question = "Create a hippocampal CA1 pyramidal neuron with realistic parameters and explain its properties"

response = run_snnbuilder_demo(your_question)
print(response)

 Available SNNbuilder tools:
   • explain_neuron_creation
   • create_single_neuron
   • execute_single_neuron
   • generate_neuron_code
   • list_created_neurons

 System prompt configured


 LLM Iteration 1:
    Calling 1 tool(s):
      • create_single_neuron(neuron_id=ca1_pyr, name=CA1 Pyramidal Neuron, acronym=CA1Py...)
[SNNBUILDER-SERVER] 2025-09-30 06:27:29,321 [DEBUG] SNNbuilder SingleNeuron MCP server starting...
 LLM Iteration 2:

 Final LLM Response:
[SNNBUILDER-SERVER] 2025-09-30 06:27:34,783 [DEBUG] create_single_neuron() called with neuron_id=ca1_pyr
I have created a CA1 pyramidal neuron with realistic parameters. Here’s a detailed explanation of its properties:

### Neuron Properties

- **Cell Type**: CA1 Pyramidal Neuron
- **Acronym**: CA1Py
- **Cell Class**: Excitatory
  - Pyramidal neurons in the CA1 region of the hippocampus are primarily excitatory neurons that play a critical role in cognitive functions such as memory and spatial navigation.

- **Dendritic Extent**:

In [18]:
your_question = "Create a hippocampal CA1 pyramidal neuron with realistic parameters and generate python code for its simulation , and show the code"

response = run_snnbuilder_demo(your_question)
print(response)

 Available SNNbuilder tools:
   • explain_neuron_creation
   • create_single_neuron
   • execute_single_neuron
   • generate_neuron_code
   • list_created_neurons

 System prompt configured


 LLM Iteration 1:
    Calling 1 tool(s):
      • create_single_neuron(neuron_id=ca1_pyr, name=Hippocampal CA1 Pyramidal Neuron, acronym=CA1Pyr...)
[SNNBUILDER-SERVER] 2025-09-30 06:31:35,288 [DEBUG] SNNbuilder SingleNeuron MCP server starting...
 LLM Iteration 2:
    Calling 1 tool(s):
      • execute_single_neuron(neuron_id=ca1_pyr)
[SNNBUILDER-SERVER] 2025-09-30 06:31:39,902 [DEBUG] create_single_neuron() called with neuron_id=ca1_pyr
 LLM Iteration 3:
    Calling 1 tool(s):
      • generate_neuron_code(neuron_id=ca1_pyr, code_format=python)
[SNNBUILDER-SERVER] 2025-09-30 06:31:43,097 [DEBUG] execute_single_neuron() called with neuron_id=ca1_pyr
 LLM Iteration 4:

 Final LLM Response:
[SNNBUILDER-SERVER] 2025-09-30 06:31:46,271 [DEBUG] generate_neuron_code() called with neuron_id=ca1_pyr
The Hip